In [ ]:
# feature extractoring and preprocessing data
import librosa,librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, scale

#Keras
import keras
from keras import models
from keras import layers
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
from keras.models import Sequential, Model, load_model
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l1
from keras.utils.vis_utils import plot_model
from sklearn import metrics
import pylab
plt.switch_backend('agg')
import itertools
import scipy as sp
from scipy import signal
import xgboost as xgb
import joblib
from glob import glob
import urllib


pd.plotting.register_matplotlib_converters()


#Loading CSV file
train_csv = pd.read_csv("/content/drive/MyDrive/CovidDataNew/cough_trial_extended.csv")
dataset = "/content/drive/MyDrive/CovidDataNew/data_new_extended.csv"

data = pd.read_csv("/content/drive/MyDrive/CovidDataNew/data_new_extended.csv")

data = data.drop(['filename'],axis=1)

genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 8  1]
 [ 2 46]]
              precision    recall  f1-score   support

           0       0.80      0.89      0.84         9
           1       0.98      0.96      0.97        48

    accuracy                           0.95        57
   macro avg       0.89      0.92      0.91        57
weighted avg       0.95      0.95      0.95        57



In [ ]:
from sklearn.model_selection import KFold # import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


In [ ]:
kf = KFold(n_splits=10)  
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator

KFold(n_splits=10, random_state=None, shuffle=False)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(confusion_matrix(y_test, y_pred))
    print(accuracy_score(y_test, y_pred))

[[ 1 12]
 [ 0  4]]
0.29411764705882354
[[ 2  4]
 [ 0 11]]
0.7647058823529411
[[ 0  0]
 [ 1 16]]
0.9411764705882353
[[ 0  0]
 [ 2 15]]
0.8823529411764706
[[ 0  0]
 [ 2 15]]
0.8823529411764706
[[17]]
1.0
[[ 0  0]
 [ 1 16]]
0.9411764705882353
[[ 0  0]
 [ 1 16]]
0.9411764705882353
[[ 0  0]
 [ 3 14]]
0.8235294117647058
[[17]]
1.0


In [ ]:
from sklearn.model_selection import  GridSearchCV
param_DT = {'criterion':['gini', 'entropy'], 'max_depth': [3,4,5,6], 'min_samples_leaf':[0.1, 0.5, 1, 1.5, 2]}
search_DT = GridSearchCV(classifier, param_DT)
search_DT.fit(X_train, y_train)
print(f'Best CV params {search_DT.best_params_}')
print(f'Best CV accuracy {search_DT.best_score_}')
print(f'Test accuracy of best hypers {search_DT.score(X_test, y_test)}')

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_leaf must be at least 1 or in (0, 0.5], got 1.5

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_leaf must be at least 1 or in (0, 0.5], got 1.5

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_leaf must be at least 1 or in (0, 0.5], got 1.5

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536:

Best CV params {'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 1}
Best CV accuracy 0.9286021505376345
Test accuracy of best hypers 1.0


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_leaf must be at least 1 or in (0, 0.5], got 1.5

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_leaf must be at least 1 or in (0, 0.5], got 1.5

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_leaf must be at least 1 or in (0, 0.5], got 1.5

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: